# WhisperX JSON segments to textgrid

> "For HSI project"

- comments: false
- branch: master
- hidden: true
- categories: [praat, textgrid, whisperx, json, hsi]

In [1]:
#%%capture
%pip install praatio

Note: you may need to restart the kernel to use updated packages.


In [2]:
from praatio import textgrid
from praatio.utilities.constants import Interval

In [3]:
input = "/Users/joregan/Playing/hsi/test2/whisperx-json"
output = '/Users/joregan/Playing/hsi/test2/whisperx-json/textgrids'

In [4]:
from pathlib import Path
import json

IS_DIR = False
input_path = Path(input)
output_path = Path(output)

if input_path.is_dir():
    IS_DIR = True
    assert not output_path.exists(), "f{output} exists, refusing to overwrite"
    output_path.mkdir()

In [5]:
def pad_silences(segments):
    start = 0.0
    out = []
    for segment in segments:
        out.append((start, segment[0], " "))
        out.append(segment)
        start = segment[1]
    return out

In [6]:
def write_converted(infile, outfile, verbose=True):
    if type(infile) == Path:
        infile = str(infile)
    if type(outfile) == Path:
        outfile = str(outfile)

    with open(infile) as inf:
        data = json.load(inf)

    out = []
    if not "segments" in data:
        if verbose:
            print("File", infile, "possibly incorrect JSON")
        return
    if len(data["segments"]) == 0:
        if verbose:
            print("File", infile, "contains no segments")
        return
    for segment in data["segments"]:
        if segment["start"] > segment["end"]:
            if verbose:
                print("File", infile, "has start time after end time")
                print(segment)
            return
        out.append(Interval(segment["start"], segment["end"], segment["text"]))

    tier_start = out[0][0]
    tier_end = out[-1][1]

    tg = textgrid.Textgrid()
    word_tier = textgrid.IntervalTier('words', pad_silences(out), tier_start, tier_end)
    tg.addTier(word_tier)
    tg.save(outfile, format="long_textgrid", includeBlankSpaces=False)

In [7]:
if not IS_DIR:
    write_converted(input, output)
else:
    for file in input_path.glob("*.json"):
        tgpath = output_path / f"{file.stem}.TextGrid"
        write_converted(file, tgpath)

File /Users/joregan/Playing/hsi/test2/whisperx-json/hsi_6_0718_222_002_inter.json contains no segments


TextgridStateError: The start time of an interval (234.743) cannot occur after its end time (232.4)